In [24]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torchvision.datasets import MNIST, CIFAR10

from utils import training, evaluation
from models import bp_model, cnn_model


# 全局配置
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CRITERION = nn.CrossEntropyLoss()
EPOCHS = 10
BATCH_SIZE = 64
LR = 1e-3

# CIFAR-10 数据加载
transform = transforms.Compose([transforms.ToTensor()])
train_ds = CIFAR10('./data', True, download=True, transform=transform)
test_ds  = CIFAR10('./data', False, download=True, transform=transform)
input_shape = train_ds[0][0].shape  # e.g. (3,32,32)

# 激活函数映射
act_map = {
    'relu': nn.ReLU(),
    'sigmoid': nn.Sigmoid(),
    'leakyrelu': nn.LeakyReLU()
}


# def run_experiment(dataset, model_type, epochs=10, batch_size=32, **model_kwargs):
#     test_accs = []
#     folds = get_folds(dataset)
#     for tr_idx, va_idx in folds:
#         tr_loader = DataLoader(Subset(train_ds_map[dataset], tr_idx), batch_size=batch_size, shuffle=True)
#         va_loader = DataLoader(Subset(train_ds_map[dataset], va_idx), batch_size=batch_size, shuffle=False)
#         # 模型构建
#         model_fn = bp_model.create_bp_model if model_type=='bp' else cnn_model.create_cnn_model
#         model = model_fn(input_shape_map[dataset], num_classes=10).to(DEVICE)
#         optimizer = optim.Adam(model.parameters(), lr=model_kwargs.get('lr', 1e-3))
#         # 训练
#         training.train_model(model, tr_loader, va_loader, epochs, DEVICE, CRITERION, optimizer)
#         # 测试
#         _, acc = evaluation.evaluate_model(model, test_loader_map[dataset], DEVICE, CRITERION)
#         test_accs.append(acc)
#     return np.array(test_accs)


def run_epoch_experiment(model_type, activation_key):
    """
    返回指定模型+激活函数下，每个 epoch 在验证集上的准确率列表
    """
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False)
    activation = act_map[activation_key]

    # 构建模型
    # if model_type == 'bp':
    #     model = CustomBPModel(input_shape, num_classes=10, activation=activation)
    # else:
    #     model = CustomCNNModel(input_shape, num_classes=10, activation=activation)
    # model.to(DEVICE)
    model_fn = bp_model.create_bp_model if model_type=='bp' else cnn_model.create_cnn_model
    model = model_fn(input_shape, num_classes=10)
    model.activation = activation

    model.to(DEVICE)

    optimizer = optim.Adam(model.parameters(), lr=LR)
    _, val_hist = training.train_model(
        model, train_loader, val_loader,
        epochs=EPOCHS, device=DEVICE,
        criterion=CRITERION, optimizer=optimizer
    )
    return val_hist['acc']

Files already downloaded and verified
Files already downloaded and verified


In [25]:
# models  = ['bp', 'cnn']
# models  = ['cnn']


activations = ['relu', 'sigmoid', 'leakyrelu']
for model_type in ['cnn']:
    plt.figure(figsize=(6,4))
    for act in activations:
        accs = run_epoch_experiment(model_type, act)
        plt.plot(range(1, EPOCHS+1), accs, marker='o', label=act.capitalize())
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Validation Accuracy', fontsize=12)
    plt.title(f'Activation Function Comparison ({model_type.upper()})', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(fontsize=10)
    plt.tight_layout()
    filename = f'activation_comparison_{model_type}.png'
    plt.savefig(filename, dpi=300)
    plt.close()
    print(f'Saved figure: {filename}')


Epoch 1/10 - Train loss: 1.4304, Train acc: 0.4899 | Val loss: 1.1884, Val acc: 0.5741
Epoch 2/10 - Train loss: 1.0460, Train acc: 0.6332 | Val loss: 0.9767, Val acc: 0.6520
Epoch 3/10 - Train loss: 0.8724, Train acc: 0.6963 | Val loss: 0.9045, Val acc: 0.6831
Epoch 4/10 - Train loss: 0.7662, Train acc: 0.7331 | Val loss: 0.8099, Val acc: 0.7235
Epoch 5/10 - Train loss: 0.6792, Train acc: 0.7630 | Val loss: 0.7996, Val acc: 0.7231
Epoch 6/10 - Train loss: 0.6107, Train acc: 0.7859 | Val loss: 0.8145, Val acc: 0.7273
Epoch 7/10 - Train loss: 0.5454, Train acc: 0.8111 | Val loss: 0.8389, Val acc: 0.7232
Epoch 8/10 - Train loss: 0.4906, Train acc: 0.8298 | Val loss: 0.8381, Val acc: 0.7249
Epoch 9/10 - Train loss: 0.4349, Train acc: 0.8484 | Val loss: 0.8326, Val acc: 0.7316
Epoch 10/10 - Train loss: 0.3847, Train acc: 0.8653 | Val loss: 0.8779, Val acc: 0.7364
Epoch 1/10 - Train loss: 1.4645, Train acc: 0.4738 | Val loss: 1.1945, Val acc: 0.5759
Epoch 2/10 - Train loss: 1.0756, Train acc